In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LinearRegression

In [ ]:
df_proxy = pd.read_csv('..\\models_data\\ibes_eps_semi_annual_by_date_2003_2024.csv')
df = pd.read_csv('..\\models_data\\stock_price_monthly_2003_2024.csv')
df = df.dropna(subset=["past_return", "past_return_skip"])

In [ ]:
start_date = '2011-01'
rebalance = 6
end_date_pd = pd.to_datetime(start_date) + pd.DateOffset(months=rebalance)
end_date = end_date_pd.strftime('%Y-%m')
end_date

# Ensure FPEDATS is in datetime format
df_proxy['FPEDATS'] = pd.to_datetime(df_proxy['FPEDATS'])

# Format to 'YYYY-MM' format
df_proxy['FPEDATS'] = df_proxy['FPEDATS'].dt.strftime('%Y-%m')
df_proxy= df_proxy.drop(columns=['Year'])

In [ ]:
# Set multi-index on ['date', 'NCUSIP'] and sort for efficient slicing
df = df.set_index(['date', 'NCUSIP'])
df.sort_index(inplace=True)

# Pre-index by date: build a dictionary with date keys for fast lookup in the formation period
dates = df.index.get_level_values('date').unique()
df_by_date = {date: df.loc[date] for date in dates}

VW = False   # Value-weighted flag
SKIP = False # Skip flag
rebalance = 6

In [ ]:
# Formation period: retrieve data for the formation period using the pre-indexed dictionary
df1 = df_by_date[start_date].copy()
sort_col = "past_return_skip" if SKIP else "past_return"
df1['rank'] = df1[sort_col].rank(method="first", ascending=False)
# Classify stocks: Winners (top 10%), Losers (bottom 10%), Neutral otherwise
df1['portfolio'] = np.where(
    df1['rank'] <= np.percentile(df1['rank'], 10), 'W',
    np.where(df1['rank'] >= np.percentile(df1['rank'], 90), 'L', 'N')
)
# Select winners and losers portfolios
buy = df1[df1['portfolio'] == 'W'].copy()
sell = df1[df1['portfolio'] == 'L'].copy()

# between start_date and end_date
df_proxy1 = df_proxy[(df_proxy['FPEDATS'] >= start_date) & (df_proxy['FPEDATS'] < end_date)].copy()

In [ ]:
print("Buy Index:", buy.index)
print("Sell Index:", sell.index)

In [ ]:
buy = buy.reset_index()
sell = sell.reset_index()
buy.sort_values('rank', inplace=True)
buy.head()

In [ ]:
# get unqiue NCUSIP and TICKER pairs
_df1 = df1.copy().reset_index()
_df1_unique = _df1[['TICKER', 'NCUSIP']].drop_duplicates()

In [ ]:
# Drop FPEDATS and FPI columns if they exist
df_proxy1 = df_proxy1.drop(columns=['FPEDATS', 'FPI'], errors='ignore')

# Merge with df_proxy1 to find missing pairs
merged = _df1_unique.merge(df_proxy1, left_on=['TICKER', 'NCUSIP'], right_on=['OFTIC', 'CUSIP'], how='left', indicator=True)

# Select missing pairs and add them with NUMEST = 0
missing_rows = merged[merged['_merge'] == 'left_only'][['TICKER', 'NCUSIP']]
missing_rows = missing_rows.rename(columns={'TICKER': 'OFTIC', 'NCUSIP': 'CUSIP'})
missing_rows['NUMEST'] = 0

# Append missing rows efficiently
df_proxy1 = pd.concat([df_proxy1, missing_rows], ignore_index=True)

# drop na
df_proxy1 = df_proxy1.dropna(subset=['OFTIC', 'CUSIP'])
df_proxy1

In [ ]:
# Ensure columns are strings and properly formatted
df_proxy1['CUSIP'] = df_proxy1['CUSIP'].astype(str).str.zfill(8)
df_proxy1['OFTIC'] = df_proxy1['OFTIC'].astype(str)

_df1['NCUSIP'] = _df1['NCUSIP'].astype(str).str.zfill(8)
_df1['TICKER'] = _df1['TICKER'].astype(str)

# Merge on CUSIP <-> NCUSIP and OFTIC <-> TICKER
df_proxy1 = df_proxy1.merge(
    _df1[['NCUSIP', 'TICKER', 'ME', 'NasdaqDummy']], 
    left_on=['CUSIP', 'OFTIC'], 
    right_on=['NCUSIP', 'TICKER'], 
    how='left'
)

# Drop redundant columns after merge
df_proxy1.drop(columns=['NCUSIP', 'TICKER'], inplace=True)
df_proxy1 = df_proxy1.dropna(subset=['ME'])

# Display result
df_proxy1


In [ ]:
# linear regression ln(1+ # of Analyst) = β0​+β1*​ln(Size​)+β2*​NasdaqDummy

# Create a new column with the natural logarithm of the number of analysts
df_proxy1['ln_analysts'] = np.log1p(df_proxy1['NUMEST'])
# Create a new column with the natural logarithm of the market capitalization
df_proxy1['ln_ME'] = np.log(df_proxy1['ME'])


# apply linear regression
X = df_proxy1[['ln_ME', 'NasdaqDummy']]
y = df_proxy1['ln_analysts']
reg = LinearRegression().fit(X, y)
beta0 = reg.intercept_
beta1, beta2 = reg.coef_


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Manually compute predicted values and ensure predictions are non-negative
y_pred = reg.predict(X)
y_pred = np.maximum(y_pred, 0)  # Ensure no negative predictions
df_proxy1["predicted_ln_analysts"] = y_pred

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)


# Visualization
plt.figure(figsize=(10, 6))

# Scatter plot of actual data points
sns.scatterplot(x=df_proxy1['ln_ME'], y=df_proxy1['ln_analysts'], hue=df_proxy1['NasdaqDummy'], alpha=0.7, palette='coolwarm')

# Regression lines
ln_ME_range = np.linspace(df_proxy1['ln_ME'].min(), df_proxy1['ln_ME'].max(), 100)

# Nasdaq regression line (orange)
ln_analysts_predicted_nasdaq = beta0 + beta1 * ln_ME_range + beta2 * 1  # NasdaqDummy = 1
plt.plot(ln_ME_range, np.maximum(ln_analysts_predicted_nasdaq, 0), color='orange', linewidth=2, label="Nasdaq Regression")

# Non-Nasdaq regression line (blue)
ln_analysts_predicted_non_nasdaq = beta0 + beta1 * ln_ME_range + beta2 * 0  # NasdaqDummy = 0
plt.plot(ln_ME_range, np.maximum(ln_analysts_predicted_non_nasdaq, 0), color='blue', linewidth=2, label="Non-Nasdaq Regression")

# Add Betas & RMSE as text on the plot
textstr = (rf"$\ln(1 + \text{{# Analysts}}) = {beta0:.4f} + {beta1:.4f} * \ln(\text{{Size}}) + {beta2:.4f} * \text{{Nasdaq}}$"
           f"\nRMSE = {rmse:.4f} \nR2 = {r2:.4f}")
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=12,
         verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

# Labels and title
plt.xlabel("ln(Size) (Market Capitalization)")
plt.ylabel("ln(1 + # of Analysts)")
plt.title(f"Linear Regression: Analysts vs. Market Capitalization {start_date} - {end_date}")
plt.legend(title="Nasdaq Dummy", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0)
plt.grid(True)
plt.show()

In [ ]:
df_proxy1["predicted_analysts"] = np.expm1(df_proxy1["predicted_ln_analysts"])
df_proxy1["residuals"] = df_proxy1["NUMEST"] - df_proxy1["predicted_analysts"]

df_proxy1.describe()

In [ ]:
df_proxy1.sort_values('residuals', ascending=False)

In [ ]:
# # merge residuals col with buy and sell
buy = buy.merge(df_proxy1[['OFTIC', 'CUSIP', 'residuals']], left_on=['NCUSIP', 'TICKER'], right_on=['CUSIP', 'OFTIC'], how='left')
sell = sell.merge(df_proxy1[['OFTIC', 'CUSIP', 'residuals']], left_on=['NCUSIP', 'TICKER'], right_on=['CUSIP', 'OFTIC'], how='left')
buy = buy.drop(columns=['CUSIP', 'OFTIC'], errors='ignore')
sell = sell.drop(columns=['CUSIP', 'OFTIC'], errors='ignore')

# drop rows with NaN residuals
buy = buy.dropna(subset=['residuals'])
sell = sell.dropna(subset=['residuals'])

In [ ]:
# long bottom 25% of residual winners
buy['portfolio'] = np.where(
    buy['residuals'] <= np.percentile(buy['residuals'], 25), 'W','N')

# short bottom 25% of residual losers
sell['portfolio'] = np.where(
    sell['residuals'] <= np.percentile(sell['residuals'], 25), 'L','N')

In [ ]:
sell.sort_values('residuals', inplace=True)
sell

In [ ]:
buy = buy[buy['portfolio'] == 'W']
sell = sell[sell['portfolio'] == 'L']

# Assign weights (equal-weighted or value-weighted)
buy.loc[:, 'weight'] = buy['ME'] / buy['ME'].sum() if VW else 1 / len(buy)
sell.loc[:, 'weight'] = sell['ME'] / sell['ME'].sum() if VW else 1 / len(sell)

buy = buy.set_index("NCUSIP")
sell = sell.set_index("NCUSIP")

In [ ]:
# Retrieve holding period data using multi-index slicing
idx = pd.IndexSlice
df_hold = df.loc[idx[start_date:end_date, :], :]
# --- Vectorized cumulative return computation ---
# Reset index so that 'NCUSIP' becomes a column
df_hold_reset = df_hold.reset_index()
# Create a new column for 1 + RET
df_hold_reset['one_plus_ret'] = 1 + df_hold_reset['RET']
# Compute cumulative product per stock over the holding period and subtract 1
accu_returns = df_hold_reset.groupby('NCUSIP')['one_plus_ret'].prod() - 1
# Compute weighted returns by aligning on stock identifier
buy_return = (buy['weight'] * accu_returns.reindex(buy.index)).sum()
sell_return = (sell['weight'] * accu_returns.reindex(sell.index)).sum()
# Overall portfolio return: long winners minus short losers
portfolio_return = buy_return - sell_return

color = '\033[92m' if portfolio_return > 0 else '\033[91m'
reset = '\033[0m'
print(f"{start_date} {end_date} {color}{round(portfolio_return * 100, 2)}%{reset}")